### Extras, Notes:

In [99]:
asdfasdfasdf

NameError: name 'asdfasdfasdf' is not defined

#### LSTM with Standardized Data

In [86]:
X = world_df.values
X = X.reshape((-1,1))

In [87]:
split_percent = 0.9
split = int(split_percent * len(X))

X_train = X[:split]
X_test = X[split:]

date_train = world_df.index[:split]
date_test = world_df.index[split:]

# scale with MinMax scaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [88]:
look_back = 3
train_gen = TimeseriesGenerator(X_train, X_train, length= look_back, batch_size= 1)
test_gen = TimeseriesGenerator(X_test, X_test, length= look_back, batch_size= 1)

In [ ]:
# set layers for LSTM model
# using tanh and leaky relu for activation
lstm_model = Sequential()
lstm_model.add(LSTM(32, activation= 'tanh', input_shape= (look_back, 1), 
                    return_sequences= True))
lstm_model.add(LSTM(64, activation= 'tanh', return_sequences= True))
lstm_model.add(LSTM(128, return_sequences= True))
lstm_model.add(LeakyReLU(alpha=0.1))
lstm_model.add(LSTM(256, return_sequences= True))
lstm_model.add(LeakyReLU(alpha=0.1))
lstm_model.add(LSTM(128, return_sequences= True))
lstm_model.add(LeakyReLU(alpha=0.1))
lstm_model.add(LSTM(64, activation= 'tanh', return_sequences= True))
lstm_model.add(LSTM(32, activation= 'tanh', return_sequences= True))
lstm_model.add(LSTM(1, activation= 'tanh'))

lstm_model.compile(optimizer= 'Adam', loss= 'mse')

lstm_model.summary()

In [90]:
lstm_model.fit(train_gen, epochs= 25)

Epoch 1/25
89/89 [==============================] - 1s 12ms/step - loss: 0.1265
Epoch 2/25
89/89 [==============================] - 1s 9ms/step - loss: 0.0479
Epoch 3/25
89/89 [==============================] - 1s 9ms/step - loss: 0.0064
Epoch 4/25
89/89 [==============================] - 1s 10ms/step - loss: 0.0047
Epoch 5/25
89/89 [==============================] - 1s 12ms/step - loss: 0.0029
Epoch 6/25
89/89 [==============================] - 1s 12ms/step - loss: 0.0018
Epoch 7/25
89/89 [==============================] - 1s 11ms/step - loss: 0.0014
Epoch 8/25
89/89 [==============================] - 1s 10ms/step - loss: 0.0012
Epoch 9/25
89/89 [==============================] - 1s 9ms/step - loss: 8.3819e-04
Epoch 10/25
89/89 [==============================] - 1s 9ms/step - loss: 8.2898e-04
Epoch 11/25
89/89 [==============================] - 1s 9ms/step - loss: 8.3185e-04
Epoch 12/25
89/89 [==============================] - 1s 9ms/step - loss: 0.0011
Epoch 13/25
89/89 [============

In [91]:
# predict and inverse_transform X data and reshape for plotting
prediction = lstm_model.predict(test_gen)

X_test = scaler.inverse_transform(X_test)
X_train = scaler.inverse_transform(X_train)

X_train = X_train.reshape((-1))
X_test = X_test.reshape((-1))

In [92]:
# reshape data to inverse transform then again to plot
prediction = prediction.reshape(prediction.shape[0], prediction.shape[1])
prediction = scaler.inverse_transform(prediction)
prediction = prediction.reshape((-1))

In [93]:
prediction

array([1.15187824e+08, 1.15443224e+08, 1.15749552e+08, 1.16041528e+08,
       1.16283104e+08, 1.16472872e+08, 1.16607784e+08, 1.16750792e+08],
      dtype=float32)

In [94]:
# plot data
trace1 = go.Scatter(
    x = date_train,
    y = X_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = X_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = 'LSTM Prediction',
    xaxis = {'title' : 'Date'},
    yaxis = {'title' : 'People Fully Vaccinated'}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [96]:
# functions to forecast model
X = X.reshape((-1))

# predict future values (fully vaccinated people)
def predict(num_prediction, model):
    prediction_list = X[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list

# create future dates for x-axis    
def predict_dates(num_prediction):
    last_date = world_df.index.values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

# num_prediction = number of days into future
num_prediction = 200
forecast = predict(num_prediction, lstm_model)
forecast_dates = predict_dates(num_prediction)

In [89]:
lstm_model.fit(train_gen, epochs= 25)

# predict and inverse_transform X data and reshape for plotting
prediction = lstm_model.predict(test_gen)

X_test = scaler.inverse_transform(X_test)
X_train = scaler.inverse_transform(X_train)

X_train = X_train.reshape((-1))
X_test = X_test.reshape((-1))

# reshape data to inverse transform then again to plot
prediction = prediction.reshape(prediction.shape[0], prediction.shape[1])
prediction = scaler.inverse_transform(prediction)
prediction = prediction.reshape((-1))

prediction

# plot data
trace1 = go.Scatter(
    x = date_train,
    y = X_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = X_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = 'LSTM Prediction',
    xaxis = {'title' : 'Date'},
    yaxis = {'title' : 'People Fully Vaccinated'}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

# functions to forecast model
X = X.reshape((-1))

# predict future values (fully vaccinated people)
def predict(num_prediction, model):
    prediction_list = X[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list

# create future dates for x-axis    
def predict_dates(num_prediction):
    last_date = world_df.index.values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

# num_prediction = number of days into future
num_prediction = 200
forecast = predict(num_prediction, lstm_model)
forecast_dates = predict_dates(num_prediction)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 3, 32)             4352      
_________________________________________________________________
lstm_10 (LSTM)               (None, 3, 64)             24832     
_________________________________________________________________
lstm_11 (LSTM)               (None, 3, 128)            98816     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 3, 128)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 3, 256)            394240    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 3, 256)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 3, 128)           

In [97]:
# plot data
trace1 = go.Scatter(
    x = world_df.index,
    y = X,
    mode = 'lines',
    name = 'Original Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Forecasted Data'
)
layout = go.Layout(
    title = 'LSTM Forecast',
    xaxis = {'title' : 'Date'},
    yaxis = {'title' : 'People Fully Vaccinated'}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

In [95]:
X_test = X_test[:8]
lstm_rmse = mean_squared_error(X_test, prediction, squared= False)
print('RMSE: {:,.2f}'.format(lstm_rmse))

RMSE: 22,047,619.91


In [ ]:
X = world_df.values

In [ ]:
y = world_df.index
y = y.strftime('%Y%m%d')
y = np.asarray(y, dtype= 'int64')

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_X = scaler.fit_transform(X)

# y = y.reshape(y.shape[0], 1)
# scaled_y = scaler.transform(y)

In [ ]:
# scaled_y

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(scaled_X, scaled_y, test_size= 0.1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size= 0.1)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(32, activation= 'relu', input_shape= (X_train.shape[1], X_train.shape[2]), 
                    return_sequences= True))
lstm_model.add(LSTM(64, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(128, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(256, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(128, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(128, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(64, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(32, activation= 'relu', return_sequences= True))
# lstm_model.add(Dropout(0.5))
# lstm_model.add(Dense(6))
# lstm_model.add(Dense(1))
lstm_model.add(LSTM(1, activation= 'relu'))
lstm_model.compile(optimizer= 'Adam', loss= 'mse')

lstm_model.summary()

In [ ]:
history = lstm_model.fit(X_train, y_train, epochs= 20, batch_size= 1, validation_data= (X_test, y_test))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
from numpy import newaxis
def predict_sequence_full(model, data, window_size):
    #Shift the window by 1 new prediction each time, re-run predictions on new window
    curr_frame = data[0]
    predicted = []
    for i in range(len(data)):
        predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
    return predicted

def plot_results(predicted_data, true_data): 
    fig = plt.figure(facecolor='white') 
    ax = fig.add_subplot(111) 
    ax.plot(true_data, label='True Data') 
    plt.plot(predicted_data, label='Prediction') 
    plt.legend() 
    plt.show() 

In [ ]:
X_test[0]

In [ ]:
y_test

In [ ]:
predictions = predict_sequence_full(lstm_model, X_test, 1)
plot_results(predictions, y_test)

In [ ]:
y_pred = lstm_model.predict(X_test)
X_test = X_test.reshape((X_test.shape[0], 1))

print(X_test.shape)
y_pred.shape

In [ ]:
print (y_pred)
# print(X_test.)

In [ ]:
X_test_inversed = scaler.inverse_transform(X_test)

In [ ]:
def plot_predictions(test,predicted):
    plt.plot(test, color='red',label='Real Data')
    plt.plot(predicted, color='blue',label='Predicted Data')
    plt.title('Real vs Predicted Data')
    plt.xlabel('Days')
    plt.ylabel('People Vaccinated')
    plt.legend()
    plt.show()

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

In [ ]:
plot_predictions(X_test_inversed, y_pred)

In [ ]:
# np.random.seed(7)
# from sklearn.preprocessing import MinMaxScaler

In [ ]:
# dataset = world_df.values
# dataset = dataset.astype('float32')

In [ ]:
# scaler = MinMaxScaler(feature_range=(0,1))
# dataset = scaler.fit_transform(dataset)

In [ ]:
# train_size = int(len(dataset) * .9)
# test_size = len(dataset) - train_size

# train, test = dataset[0: train_size, :], dataset[train_size: len(dataset), :]
# print(len(train), len(test))

In [ ]:
# def create_dataset(dataset, look_back=1):
#     dataX, dataY = [], []
#     for i in range(len(dataset)-look_back-1):
#         a = dataset[i:(i+look_back), 0]
#         dataX.append(a)
#         dataY.append(dataset[i + look_back, 0])
#     return np.array(dataX), np.array(dataY)

In [ ]:
# look_back = 1
# trainX, trainY = create_dataset(train, look_back)
# testX, testY = create_dataset(test, look_back)

In [ ]:
# trainX

In [ ]:
# reshape input to be [samples, time steps, features]
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# lstm_model = Sequential()
# lstm_model.add(LSTM(32, activation= 'relu', input_shape= (1, 1), return_sequences= True))
# lstm_model.add(LSTM(64, activation= 'relu', return_sequences= True))
# lstm_model.add(LSTM(128, activation= 'relu', return_sequences= True))
# lstm_model.add(LSTM(64, activation= 'relu', return_sequences= True))
# lstm_model.add(LSTM(32, activation= 'relu', return_sequences= True))
# # lstm_model.add(Dropout(0.5))
# # lstm_model.add(Dense(6))
# # lstm_model.add(Dense(1))
# lstm_model.add(LSTM(1, activation= 'relu'))
# lstm_model.compile(optimizer= 'Adam', loss= 'mse')

# lstm_model.summary()

In [ ]:
# lstm_model.fit(trainX, trainY, epochs=100, batch_size= 1)

In [ ]:
# testY

In [ ]:
# make predictions
# trainPredict = lstm_model.predict(trainX)
# testPredict = lstm_model.predict(testX)
# # invert predictions
# trainPredict = scaler.inverse_transform(trainPredict)
# trainY = scaler.inverse_transform([trainY])
# testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform([testY])
# # calculate root mean squared error
# trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
# print('Train Score: %.2f RMSE' % (trainScore))
# testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
# print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# testPredict

In [ ]:
# shift train predictions for plotting
# trainPredictPlot = np.empty_like(dataset)
# trainPredictPlot[:, :] = np.nan
# trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# # shift test predictions for plotting
# testPredictPlot = np.empty_like(dataset)
# testPredictPlot[:, :] = np.nan
# testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# # plot baseline and predictions
# plt.figure(figsize= (12, 7))
# plt.plot(scaler.inverse_transform(dataset), label= 'original data')
# plt.plot(trainPredictPlot, label= 'pred train')
# plt.plot(testPredictPlot, label= 'pred test')
# plt.legend()
# plt.show()

In [ ]:
# scale the data
# scaler = StandardScaler()
# train_arr = scaler.fit_transform(train)
# test_arr = scaler.transform(test)

In [ ]:
day_groups = world_df.groupby(pd.Grouper(freq= 'D'))

daily_vax = pd.concat([pd.DataFrame(x[1].values) for x in day_groups], axis= 1)
daily_df = pd.DataFrame(daily_vax)

#### Newer data:

In [ ]:
fully_vax_df2 = pd.read_csv('../data/people-fully-vaccinated-covid2.csv')
fully_vax_df2['Day'] = pd.to_datetime(fully_vax_df2['Day'])

fully_vax_ts2 = fully_vax_df2.copy()
fully_vax_ts2.set_index('Day', inplace= True)

world_df2 = fully_vax_ts2.where(fully_vax_ts2['Entity'] == 'World')
world_df2.dropna(inplace= True)
world_df2.drop(['Code', 'Entity'], axis= 1, inplace= True)

In [ ]:
world_df2

In [ ]:
train, test = train_test_split(world_df2, test_size= 0.1)

In [ ]:
print(train.shape)
test.shape

In [ ]:
model2 = pm.auto_arima(train)

In [ ]:
preds, conf_int = model2.predict(n_periods=test.shape[0], return_conf_int=True)
print('Test RMSE: %.3f' % mean_squared_error(test, preds, squared= False))

In [ ]:
x = np.arange(world_df2.shape[0])
plt.figure(figsize= (8, 6))
plt.plot(x[:74], train, c='blue', label= 'Training Data')
plt.plot(x[74:], preds, c='red', label= 'Predicted Data')
plt.plot(x[74:], test, c= 'green', label= 'Test Data')
plt.title('Predicted Vaccinations Vs Actual Vaccinations')
plt.xlabel('Days')
plt.ylabel('Vaccinations')
plt.legend()
plt.show()

In [ ]:
daily_df.boxplot(figsize= (15, 10))

In [ ]:
# data_diff = world_df.diff().dropna()
# data_diff

In [ ]:
# arima_world = ARIMA(data_diff, order= (3, 0, 3))
# arima_world.fit()

In [ ]:
arima_world = ARIMA(ts_log_decompose, order= (3, 0, 3))
arima_world.fit()

In [ ]:
# arima_world.param_names

### Model using Forecast function

In [ ]:
len(world_df)

In [ ]:
# pip install --upgrade ipykernel

In [ ]:
split_point = len(world_df) - 5
data, validation = world_df[0:split_point], world_df[split_point:]
print('Data %d, Validation %d' % (len(data), len(validation)))

In [ ]:
X = differencing.values
X

In [ ]:
model = ARIMA(X, order= (2, 0, 1))

In [ ]:
model_fit = model.fit()
model_fit.summary()

In [ ]:
prediction = model_fit.predict(start= 60, end= 73, dynamic= True)

In [ ]:
plt.plot(prediction)

In [ ]:
forecast = model_fit.forecast(steps= 21)[0]

In [ ]:
forecast

In [ ]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [ ]:
history = [x for x in X]
days = 60
day = 1
for yhat in forecast:
    inverted = inverse_difference(history, yhat, days)
    print('Day %d: %f' % (day, inverted))
    history.append(inverted)
    day +=1

In [ ]:
plt.plot(forecast)
plt.show()


In [ ]:
# split test data into test and validation
# validation = test[:4]
# test = test[4:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_scaler = MinMaxScaler()

scaled_train = min_scaler.fit_transform(train)
# scaled_val = min_scaler.transform(validation)
scaled_test = min_scaler.transform(test)

In [ ]:
scaled_train.shape

In [ ]:
n_input = 70
n_features = 1

generator = TimeseriesGenerator(scaled_train, scaled_train, length=1, batch_size= 4)

In [ ]:
lstm_predictions_scaled = list()

batch = scaled_train[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[lstm_pred]], axis= 1)

In [ ]:
lstm_predictions = min_scaler.inverse_transform(lstm_predictions_scaled)
lstm_predictions

In [ ]:
ewm_daily = world_df['people_fully_vaccinated'].ewm(span= 3).mean()
ewm_pred = ewm_daily.resample('D').first()
ewm_pred = ewm_pred.shift(1).dropna()
print(ewm_pred.iloc[-1])

In [ ]:
# add_dates = [world_df.index[-1] + DateOffset(days= x) for x in range(0, 9)]
# future_dates = pd.DataFrame(index= add_dates[1:], columns= world_df.columns)

# df_predict = pd.DataFrame(lstm_predictions, index= future_dates[-n_input:].index, 
# columns= ['Prediction'])
# df_proj = pd.concat([world_df, df_predict], axis= 1)

In [ ]:
# import statsmodels.tsa.ar_model.AutoRegResults.plot_predict
# model.plot_diagnostics(figsize =(8, 5))

In [ ]:
# X = np.vstack((world_df['people_fully_vaccinated'], ewm_daily))

# y = len(world_df.index)

# from sklearn import linear_model
# clf = linear_model.LinearRegression()
# clf.fit(X, y)
# print(clf.coef_)

In [ ]:
lstm_predictions_scaled = list()


batch = train_arr[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[lstm_pred]], axis= 1)

In [ ]:
lstm_predictions_scaled = list()
n_input = X_train.shape[0]
n_features = 1

batch = scaled_X[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(y)):
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[lstm_pred]], axis= 1)

In [ ]:
lstm_predictions_scaled

In [ ]:
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
lstm_predictions

In [ ]:
plt.plot(lstm_predictions)

In [ ]:
#invert scaling for forecast
inv_ypred = np.concatenate((y_pred, X_test), axis= 1)
inv_ypred = scaler.inverse_transform(inv_ypred)
inv_ypred = inv_ypred[:, 0]

# invert scaling for actual data
y_test = y_test.reshape((len(y_test), 1))
inv_y = np.concatenate((y_test, X_test), axis= 1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:, 0]

# calculate RMSE
lstm_rmse = mean_squared_error(inv_y, inv_ypred, squared= False)
print('RMSE: {:,.2f}'.format(lstm_rmse))

In [ ]:
plt.plot(inv_y)

In [ ]:
inv_ypred

In [ ]:
plt.plot(y_pred)

In [ ]:
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler_train = scaler.fit_transform(X_train)
# scaler_test = scaler.transform(X_test)

In [ ]:
train, test = train_test_split(world_df, test_size= 0.1)

In [ ]:
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler = scaler.fit(train)

In [ ]:
# scale the data
scaler = StandardScaler()
train_arr = scaler.fit_transform(train)
test_arr = scaler.transform(test)

In [ ]:
# # transform train
# train = np.asarray(train)
# train = train.reshape(train.shape[0], train.shape[1])
# train_scaled = scaler.transform(train)
# # transform test
# test = np.asarray(test)
# test = test.reshape(test.shape[0], test.shape[1])
# test_scaled = scaler.transform(test)

In [ ]:
# print(train_arr)
# train_scaled

In [ ]:
x_train = train_arr.reshape(-1, 1, 1)
x_test  = test_arr.reshape(-1, 1, 1)

In [ ]:
x_train.shape

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(32, activation= 'relu', input_shape= (x_train.shape[1], x_train.shape[2]), return_sequences= True))
lstm_model.add(LSTM(64, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(128, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(64, activation= 'relu', return_sequences= True))
lstm_model.add(LSTM(32, activation= 'relu', return_sequences= True))
# lstm_model.add(Dropout(0.5))
# lstm_model.add(Dense(6))
# lstm_model.add(Dense(1))
lstm_model.add(LSTM(1, activation= 'relu'))
lstm_model.compile(optimizer= 'Adam', loss= 'mse')

lstm_model.summary()

In [ ]:
lstm_fitted = lstm_model.fit(train_arr, test_arr, epochs= 15, batch_size= 1)

In [ ]:
losses_lstm = lstm_model.history.history['loss']
plt.figure(figsize= (12,4))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(np.arange(0,30,1))
plt.plot(range(len(losses_lstm)),losses_lstm);

In [ ]:
lstm_predictions_scaled = list()

batch = train_arr[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[lstm_pred]], axis= 1)

In [ ]:
lstm_predictions_scaled

In [ ]:
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
lstm_predictions

In [ ]:
plt.plot(lstm_predictions)

In [ ]:
test2 = test.copy()
test2['LSTM Predictions'] = lstm_predictions

In [ ]:
test2

In [ ]:
test2['people_fully_vaccinated'].plot(figsize= (16,5), legend= True)
test2['LSTM Predictions'].plot(legend= True)
plt.show()

In [ ]:
# calculate RMSE
lstm_rmse = mean_squared_error(test, lstm_predictions)
print('RMSE: {:,.2f}'.format(lstm_rmse))

In [ ]:
# calculate Mean Absolute Percentage Error
lstm_mape = round(np.mean(np.abs(lstm_predictions - test.values)/np.abs(test.values))*100, 3)  
print('MAPE:', sm_mape)

Predict Next 8 Days

In [ ]:
add_dates = [world_df.index[-1] + DateOffset(days= x) for x in range(0, 9)]
future_dates = pd.DataFrame(index= add_dates[1:], columns= world_df.columns)

df_predict = pd.DataFrame(lstm_predictions, index= future_dates[-n_input:].index, 
columns= ['Prediction'])
df_proj = pd.concat([world_df, df_predict], axis= 1)

In [ ]:
# Change format of y labels to billions
def millions(x, pos):
    'The two args are the value and tick position'
    return '%1.fM' % (x * 1e-6)
formatter = FuncFormatter(millions)

fig, ax = plt.subplots(figsize= (15,5))
ax.yaxis.set_major_formatter(formatter)
plt.plot(df_proj.index, df_proj['people_fully_vaccinated'])
plt.plot(df_proj.index, df_proj['Prediction'], color= 'r')
plt.title('LSTM Prediction for Next 8 Days')
plt.xlabel('Days')
plt.ylabel('Fully Vaccinated People')
plt.legend(labels= ['Fully Vaccinated', 'Predicted'])
plt.show()

In [ ]:
# lstm_rmse = rmse(test2['people_fully_vaccinated'], test2['LSTM Predictions'])
# print('LSTM RMSE: {:,.2f}'.format(lstm_rmse))